In [1]:
import importlib
import pandas as pd
import numpy as np
import re

from functions.functions_dataframe import * 
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

pd.set_option("display.max_colwidth", 200)

###### Step 1. Завантажити датафрейми ProZorro, МОН, створити список ЄДРПОУ освітніх закладів

In [13]:
"""
Завантажити базу закупівель ProZorro (деталі у README.txt)
"""
df_prozorro = pd.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep = ';', low_memory = False)

In [14]:
"""
Завантажити датафрейм МОН навчальниз закладів, підготовлений у "fileter MON data.ipynb"
"""
df_mon_data_bez_upravlin_osvity = pd.read_csv('data/MON_schools_dnz.csv', sep = ';')

In [15]:
"""
Створити список унікальних ЄДРПОУ освітніх закладів МОН
"""
mon_data_list = df_mon_data_bez_upravlin_osvity.edrpou.unique().tolist()

### ШКОЛИ ЗАМОВНИКИ

##### Step 2: Створити базу навчальних закладів, які самостійно купують через ProZorro

In [16]:
"""
Відфільтрувати базу закупівель df_prozorro за колонкою IDOrganizator.
Фільтром є списко МОН з ЄДРПОУ начальних закладів mon_data_list (попередній крок)

df_tenders_by_schools - база закупівель, де навчальні заклади є замовниками
df_undefined_1 - база невизначених закупівель ProZorro
"""
df_tenders_by_schools = df_prozorro[df_prozorro['IDOrganizator'].isin(mon_data_list)]
df_undefined_1 = df_prozorro[~df_prozorro['IDOrganizator'].isin(mon_data_list)]

##### Step 3: Класифікація шкіл-замовників по типах за колонкою "Организатор" та нумерація

In [17]:
df_tenders_by_schools_classified = classification_by_type_one_column(df_tenders_by_schools, ['Организатор'], filter_priority, filter_name_priority, 'Тип закладу')
df_tenders_by_schools_numerated = numeration_one_column(df_tenders_by_schools_classified, ['Организатор'], reg_number)

/home/artem/projects/school-procurements/functions/functions_dataframe.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  given_df[new_column] = "Невідомо"
/home/artem/projects/school-procurements/functions/functions_dataframe.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  given_df[column + "_check"] = ""
/home/artem/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **

##### Step 4: Cтворення фінальної бази ШКІЛ ЗАМОВНИКІВ

In [18]:
"""
"Назва закладу" - колонка з фіналізованою назвою школи. Принцип визначення:
    1. Якщо колонка "Номер" не є пустою - додається до "Типу закладу"
    2. Якщо колонка "Номер" є пустою - "Назва закладу" отримує значення "Организатор"
    
* колонка "Назва" не враховується через високий ступінь повторюваності з "Организатор" і часткову точність
"""
df_tenders_by_schools = df_tenders_by_schools_numerated.copy()
df_tenders_by_schools['Номер'] = df_tenders_by_schools['Номер'].replace(r'^\s*$', np.nan, regex=True) # технічна дія: перетворити пусті строки в NaN
df_tenders_by_schools['Назва закладу'] = np.where(df_tenders_by_schools['Номер'].notnull(), 
                                                  df_tenders_by_schools['Тип закладу'] + ' ' + df_tenders_by_schools['Номер'], 
                                                  df_tenders_by_schools['Организатор'])

In [19]:
"""
Створення колонки 'Заклад-замовник'
Видалення "технічних" колонок, створених функціями.
"""
df_tenders_by_schools['Заклад-замовник'] = True
col_names = df_prozorro.columns.tolist()
col_names += ['Заклад-замовник', 'Назва закладу']
df_tenders_by_schools = df_tenders_by_schools[col_names]

### ШКОЛИ НЕ-ЗАМОВНИКИ

##### Step 5: Пошук закупівель шкіл-не-замовників за допомогою ключових слів

In [20]:
"""
З-поміж датафрейму невизначених закупівель (df_undefined_1) за ключовими словами у списках filter_keywords, filter_keywords_strict 
відбираються закупівлі освітніх закладів у колонках: Тендер, ОписаниеТендера, Лот 

Регулярний вираз /b{}/b шукає за ключовими словами зі списку filter_keywords_strict_extended тільки 
ті паттерни, де ці ключові слова є окремими, а не зустрічають всередині слів. 
Це зменшить кількість "брудних" даних
"""
df_tenders_not_by_schools_by_keywords = df_undefined_1[df_undefined_1['Тендер'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True)  | 
                                                      df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True) | 
                                                      df_undefined_1['Лот'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True) | df_undefined_1['Тендер'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True) | df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True) | df_undefined_1['Лот'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True)]

##### Step 6: Класифікація шкіл-не-замовників за типами

In [21]:
df_tenders_not_by_schools_classified = classification_by_type_three_cols(df_tenders_not_by_schools_by_keywords,
                                                                         ['Тендер', 'ОписаниеТендера', 'Лот'], 
                                                                         filter_priority, 
                                                                         filter_name_priority)

/home/artem/projects/school-procurements/functions/functions_dataframe.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  given_df['Тип закладу'] = "Невідомо"
/home/artem/projects/school-procurements/functions/functions_dataframe.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  given_df[column + "_check"] = ""
/home/artem/projects/school-procurements/functions/functions_dataframe.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

##### Step 7: Нумерація шкіл-не-замовників

In [22]:
"""
df_step7 - датафрейм, створений із закупівель df_tenders_not_by_schools_classified, де був успішно визначений "Тип закладу"
"""
df_step7 = df_tenders_not_by_schools_classified[df_tenders_not_by_schools_classified['Тип закладу'] != 'Невідомо'].copy()

In [23]:
"""
Функція long_numeration створює два дафайреми:
    df_tenders_not_by_schools_numerated - тендери, де номер визначився 
    df_step7_numerated_undefined - тендери, де номер не визначився
"""
df_tenders_not_by_schools_numerated, df_step7_numerated_undefined = long_numeration(df_step7, reg_number)
df_tenders_not_by_schools_numerated['Назва закладу'] = df_tenders_not_by_schools_numerated['Тип закладу'] + ' ' + df_tenders_not_by_schools_numerated['Номер']

##### Step 8: Найменування шкіл-не-замовників

In [24]:
"""
Завантажити датафрейм МОН власних назв навчальниз закладів, підготовлений у "fileter MON data.ipynb"
school_names - список унікальних назв 
"""
df_names_clean = pd.read_csv('data/MON_names.csv', sep = ';')
df_names_clean = df_names_clean[df_names_clean['Назва'].notnull()] # Видалити NaN
school_names = df_names_clean['Назва'].unique().tolist()

In [25]:
"""
Функція long_naming створює два дафайреми:
    df_tenders_not_by_schools_named - тендери, де номер визначився.
    df_tenders_not_by_schools_undefined - тендери шкіл не замовників, де не визначилась назва (і номер).
    Такі закупівлі вважаються закупівлями управінь освіти для різних закладів.
"""
df_tenders_not_by_schools_named, df_tenders_not_by_schools_undefined = long_naming(df_step7_numerated_undefined, reg_double_quotes, school_names)
df_tenders_not_by_schools_named['Назва закладу'] = df_tenders_not_by_schools_named['Тип закладу'] + ' ' + df_tenders_not_by_schools_named['Назва']

/home/artem/projects/school-procurements/functions/functions_dataframe.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_named_undefined_1['Назва'] = np.where(df_named_undefined_1['Лот_naming_first'].astype(str).str.contains(r'^({})$'.format('|'.join(school_names)), case = False, na = False, regex = True), df_named_undefined_1['Лот_naming_first'], "Невідомо")
/home/artem/projects/school-procurements/functions/functions_dataframe.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_named_undefined

##### Step 9: Об'єднання двох датафреймів шкіл-не-замовників

In [26]:
"""
df_tenders_not_by_schools - датафрейм шкіл-не-замовників з визначеними номерами і назвами
Створення колонки 'Заклад-замовник'
"""
frames = [df_tenders_not_by_schools_numerated, df_tenders_not_by_schools_named]
df_tenders_not_by_schools = pd.concat(frames)
df_tenders_not_by_schools['Заклад-замовник'] = False
df_tenders_not_by_schools = df_tenders_not_by_schools[col_names]

In [ ]:
df_tenders_not_by_schools_undefined.sample(50)

##### Step 10. Підготовка датафрейму з координатами

In [27]:
"""
df_coordinates - база даних шкіл МОН з 24 містами обласних центрів України, включно з координатами
"""
df_coordinates = pd.read_csv('data/coordinates/coordinates_DNZ _SC.csv', sep=';')

In [28]:
"""
    B df_coordinates cтворити нову колонку ЄДРПОУ без помилкових текстових символів
"""
df_coordinates['edrpou_clean'] = df_coordinates.edrpou.str.extract('(\d+)', expand=False)
df_coordinates['edrpou_clean'] = df_coordinates['edrpou_clean'].fillna(0)
df_coordinates['edrpou_clean'] = pd.to_numeric(df_coordinates['edrpou_clean'])
df_coordinates['edrpou_clean'] = df_coordinates['edrpou_clean'].astype(np.int64)

In [29]:
"""
    Заповнення відсутніх даних в базі МОН: заміна стрінги '(null)' на "ДНЗ"
"""
df_coordinates['updated_education_type_name'] = np.where(df_coordinates.education_type_name == '(null)',
                                              'ДНЗ', df_coordinates.education_type_name)

In [30]:
"""
    Пошук і визначення типу закладу в колонках 'university_name' та 'updated_education_type_name'
"""
df_coordinates = classification_by_type_one_column(df_coordinates, ['university_name'], filter_priority, filter_name_priority, 'Тип закладу 1')
df_coordinates = classification_by_type_one_column(df_coordinates, ['updated_education_type_name'], filter_priority, filter_name_priority, 'Тип закладу 2')
df_coordinates['Тип закладу'] = np.where(~df_coordinates['Тип закладу 1'].str.contains('Невідомо'),
                                           df_coordinates['Тип закладу 1'],
                                           df_coordinates['Тип закладу 2'])
df_coordinates['Тип закладу'] = np.where(~df_coordinates['Тип закладу'].str.contains('Невідомо'),
                                           df_coordinates['Тип закладу'],
                                           df_coordinates['updated_education_type_name'])
df_coordinates['Тип закладу'] = df_coordinates['Тип закладу'].str.replace('загальносвітня школа', 'Загальносвітня школа')

In [31]:
"""
    df_coordinates_numerated - виокремлення номерів шкільних закладів з бази МОН 
"""
df_coordinates_numerated = numeration_one_column(df_coordinates, ['university_name'], reg_number)

In [32]:
"""
    df_coordinates_named - виокремлення власні назви шкільних закладів з бази МОН.
    Створення колонки "id" з визначених двома способами власної назви закладу
"""
df_coordinates_named = naming(df_coordinates, ['university_name'], reg_double_quotes, "naming")
df_coordinates_named['Назва'] = np.where(df_coordinates_named.university_name_naming_first.notnull(),
                                             df_coordinates_named.university_name_naming_first,
                                             df_coordinates_named.university_name_naming_last)
"""
    Створення колонки "Назва закладу". Там, де є номер - присвоєюється номер, в інших випадках - власна назва
"""
df_coordinates_named['Назва закладу'] = np.where(df_coordinates_named['Номер'].str.contains('№'), 
                                               df_coordinates['Тип закладу'] + ' '+ df_coordinates_named['Номер'],
                                               df_coordinates['Тип закладу'] + ' '+ df_coordinates_named['Назва'])
df_coordinates_named['id'] =  df_coordinates_named.koatuu_name_short + ' ' + df_coordinates_named['Назва закладу']

In [33]:
"""    
    Створення датафрейму df_coordinates_with_id з id як назвою закладу та унікальним ідентифікатором
"""

cols_to_include = ['id', 'Longitude', 'Latitude', 'edrpou_clean']
df_coordinates_with_id = df_coordinates_named[cols_to_include]
df_coordinates_with_id['id'] = df_coordinates_with_id['id'].str.replace('№ ', '№')

/home/artem/.local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


##### Step 11: Об'єднання датафреймів шкіл-не-замовників та шкіл-замовників

In [34]:
"""
df_tenders - Тендери з новими конолоками: "city", "id"
"""
frames = [df_tenders_by_schools, df_tenders_not_by_schools]
df_tenders = pd.concat(frames)
df_tenders = classification_by_type_one_column(df_tenders, ['Главный орган'], golovuj_organ_list, cities_list, 'city')
df_tenders['id'] = df_tenders['city'] + ' ' + df_tenders['Назва закладу']
df_tenders['id'] = df_tenders['id'].str.replace('№ ', '№')

##### Step 12: Створення готового датафрейму шкіл-замовників

In [35]:
"""
df_tenders_true - тендери шкіл-замовників
"""
df_tenders_true = df_tenders[df_tenders['Заклад-замовник'] == True]
df_tenders_true.id = np.where(df_tenders_true.id.str.contains('Невідомо'), 
                              df_tenders_true.id.str.replace('Невідомо', filter_shkola_name), 
                              df_tenders_true.id)

/home/artem/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [36]:
"""
tenders_by_schools - тендери шкіл-замовників з координатами і містом
"""
tenders_by_schools_completed = df_tenders_true.merge(df_coordinates_with_id, left_on='IDOrganizator', right_on='edrpou_clean')
tenders_by_schools_completed['id'] = np.where(tenders_by_schools_completed.id_x.str.contains('Невідомо'),
                                  tenders_by_schools_completed.id_y,
                                  tenders_by_schools_completed.id_x)

In [82]:
tenders_by_schools_completed_2 = tenders_by_schools_completed.drop_duplicates(subset=['ИдентификаторЛота'])
tenders_by_schools_completed_2.shape

(30514, 17)

In [83]:
tenders_by_schools_completed.shape

(33498, 17)

###### Step 13. Створення датафрейму шкіл-не-замовників

In [37]:
"""
df_tenders_false - тендери шкіл-не-замовників
"""
df_tenders_false = df_tenders[df_tenders['Заклад-замовник'] == False]

In [38]:
"""
    tenders_not_by_schools - фінальна таблиця шкіл-не-замовників
"""
tenders_not_by_schools = pd.merge(df_tenders_false, 
                                  df_coordinates_with_id, 
                                  on='id', 
                                  how='left')

In [ ]:
# tenders_not_by_schools[tenders_not_by_schools.Latitude.isnull()]
# len(tenders_not_by_schools[tenders_not_by_schools.Latitude.isnull()].id.unique().tolist())
# tenders_not_by_schools[tenders_not_by_schools.Latitude.isnull()].sample(3)

##### Step 14. Визначення координат Google Geolocator API у школах за назвою закладу в колонці 'id'

In [42]:
tenders_without_coordinates = tenders_not_by_schools[tenders_not_by_schools.Latitude.isnull()]
tenders_without_coordinates.drop(['Longitude', 'Latitude'], axis=1, inplace = True)

/home/artem/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [43]:
"""
    tenders_with_coordinates - датафрейм закупівель шкіл-не-замовників, 
    де не визначились координати раніше
"""

unique_school_names = tenders_without_coordinates.id.unique().tolist()
df_unique_school_names = pd.DataFrame(unique_school_names, columns =['id'])

# df_with_coordinates = get_coordinates(df_unique_school_names, 'id', 'Address')
# df_with_coordinates.to_excel("data/coordinates/df_with_coordinates.xlsx", index=False)
# df_with_coordinates.to_csv("data/coordinates/df_with_coordinates.csv", index=False, sep=';')

df_with_coordinates = pd.read_csv('data/coordinates/df_with_coordinates.csv', sep=';')

tenders_with_coordinates = pd.merge(tenders_without_coordinates, 
                                  df_with_coordinates, 
                                  on='id', 
                                  how='left')

##### Step 15: Створення готового датафрейму шкіл-не-замовників

In [44]:
tenders_not_by_schools_completed = pd.concat([tenders_with_coordinates, tenders_not_by_schools[tenders_not_by_schools.Latitude.notnull()]])

##### Step 16: Створення готового датафрейму всіх навчальних закладів

In [45]:
final_talbe_columns = ['id', 'city', 'Назва закладу', 'ИдентификаторЛота', 'Идентификатор', 'Организатор', 'Тендер',
       'СуммаЛота', 'ОписаниеТендера', 'Лот', 'Адрес поставки',
       'АдресОрганизатора', 'Главный орган', 'IDOrganizator',
       'Заклад-замовник', 'Longitude', 'Latitude']

tenders_by_schools_completed = tenders_by_schools_completed[final_talbe_columns]
tenders_not_by_schools_completed = tenders_not_by_schools_completed[final_talbe_columns]

final_df = pd.concat([tenders_by_schools_completed, tenders_not_by_schools_completed])

### DEBUG COORDINATES

In [76]:
final_df.shape

(53577, 17)

In [47]:
final_df.Longitude.value_counts()

30.6153935           608
30.42124179999999    542
34.9180679           404
34.77950149999999    310
27.0172632           298
                    ... 
30.3672618             1
35.221224              1
24.0039323             1
30.54922449999999      1
23.9891794             1
Name: Longitude, Length: 3702, dtype: int64

In [49]:
final_df.columns

Index(['id', 'city', 'Назва закладу', 'ИдентификаторЛота', 'Идентификатор',
       'Организатор', 'Тендер', 'СуммаЛота', 'ОписаниеТендера', 'Лот',
       'Адрес поставки', 'АдресОрганизатора', 'Главный орган', 'IDOrganizator',
       'Заклад-замовник', 'Longitude', 'Latitude'],
      dtype='object')

In [57]:
final_df[final_df['ИдентификаторЛота'].str.contains('UA-2018-08-23-000199-b-L1')]

,id,city,Назва закладу,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator,Заклад-замовник,Longitude,Latitude
1710,Запоріжжя ДНЗ №1849,Запоріжжя,ДНЗ № 1849,UA-2018-08-23-000199-b-L1,UA-2018-08-23-000199-b,Територіальний відділ освіти Заводського району департаменту освіти і науки Запорізької міської ради,"Послуги з розробки проектно-кошторисної документації для об""єкта: заклад дошкільної освіти (ясла-садок) № 256 ""Пролісок"" (код ДК 021:2015 - 71320000-7 Послуги з інженерного проектування), а саме р...",50294.36,"додаток до річного плану, додаткові кошти, довідка № 1849 від 14.08.2018","Послуги з розробки проектно-кошторисної документації для об""єкта: заклад дошкільної освіти (ясла-садок) № 256 ""Пролісок"" (код ДК 021:2015 - 71320000-7 Послуги з інженерного проектування), а саме р...","вул.Історична,30","вул. Радіаторна, 49",Запорізька міська рада,37611443,False,35.2227,47.8628


In [59]:
final_df[final_df['Longitude'].str.contains('30.6153935', na=False)]

,id,city,Назва закладу,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator,Заклад-замовник,Longitude,Latitude
4849,Київ Навчально-виховний комплекс (об'єднання) №204,Київ,Навчально-виховний комплекс (об'єднання) №204,UA-2017-01-05-000548-c-L1,UA-2017-01-05-000548-c,Навчально-виховний комплекс «Домінанта» (дошкільний навчальний заклад - спеціалізована школа І ступеня - суспільно-гуманітарна гімназія – СШ №204 з поглибленим вивченням фізичної культури),Яловичина без кістки (охолоджена),170000,"Детальний опис предмета закупівлі, технічні вимоги, якісні характеристики та зразок заповнення договору знаходяться в тендерній документації (файл Відкорегована документація 15111100-0 Яловичина)",Яловичина без кістки (охолоджена),"Юності, 3, Юності, 1, Бойченка, 16А","вулиця Юності, будинок, 3",КМДА,22875220,True,30.6153935,50.46384579999999
4850,Київ Навчально-виховний комплекс (об'єднання) №204,Київ,Навчально-виховний комплекс (об'єднання) №204,UA-2017-01-05-000548-c-L1,UA-2017-01-05-000548-c,Навчально-виховний комплекс «Домінанта» (дошкільний навчальний заклад - спеціалізована школа І ступеня - суспільно-гуманітарна гімназія – СШ №204 з поглибленим вивченням фізичної культури),Яловичина без кістки (охолоджена),170000,"Детальний опис предмета закупівлі, технічні вимоги, якісні характеристики та зразок заповнення договору знаходяться в тендерній документації (файл Відкорегована документація 15111100-0 Яловичина)",Яловичина без кістки (охолоджена),"Юності, 3, Юності, 1, Бойченка, 16А","вулиця Юності, будинок, 3",КМДА,22875220,True,30.6153935,50.46384579999999
4851,Київ Навчально-виховний комплекс (об'єднання) №204,Київ,Навчально-виховний комплекс (об'єднання) №204,UA-2017-05-11-001428-b-L1,UA-2017-05-11-001428-b,Навчально-виховний комплекс «Домінанта» (дошкільний навчальний заклад - спеціалізована школа І ступеня - суспільно-гуманітарна гімназія – СШ №204 з поглибленим вивченням фізичної культури),Кухонне приладдя (посуд столовий),22000,"Детальний опис предмета закупівлі, технічні та якісні характеристики, кількість, а також проект договору знаходяться в тендерній документації",Кухонне приладдя (посуд столовий),"Юності, 3","вулиця Юності, будинок, 3",КМДА,22875220,True,30.6153935,50.46384579999999
4852,Київ Навчально-виховний комплекс (об'єднання) №204,Київ,Навчально-виховний комплекс (об'єднання) №204,UA-2017-05-11-001428-b-L1,UA-2017-05-11-001428-b,Навчально-виховний комплекс «Домінанта» (дошкільний навчальний заклад - спеціалізована школа І ступеня - суспільно-гуманітарна гімназія – СШ №204 з поглибленим вивченням фізичної культури),Кухонне приладдя (посуд столовий),22000,"Детальний опис предмета закупівлі, технічні та якісні характеристики, кількість, а також проект договору знаходяться в тендерній документації",Кухонне приладдя (посуд столовий),"Юності, 3","вулиця Юності, будинок, 3",КМДА,22875220,True,30.6153935,50.46384579999999
4853,Київ Навчально-виховний комплекс (об'єднання) №204,Київ,Навчально-виховний комплекс (об'єднання) №204,UA-2017-10-11-001674-b-L1,UA-2017-10-11-001674-b,Навчально-виховний комплекс «Домінанта» (дошкільний навчальний заклад - спеціалізована школа І ступеня - суспільно-гуманітарна гімназія – СШ №204 з поглибленим вивченням фізичної культури),"Поточний ремонт, утеплення фасаду суспільно-гуманітарної гімназії НВК ""Домінанта"" управління освіти Дніпровської районної в місті Києві державної адміністрації за адресою: вул.Юності,1",250000,"Поточний ремонт, утеплення фасаду суспільно-гуманітарної гімназії НВК ""Домінанта"" управління освіти Дніпровської районної в місті Києві державної адміністрації за адресою: вул.Юності,1","Поточний ремонт, утеплення фасаду суспільно-гуманітарної гімназії НВК ""Домінанта"" управління освіти Дніпровської районної в місті Києві державної адміністрації за адресою: вул.Юності,1","вул.Юності,1","вулиця Юності, будинок, 3",КМДА,22875220,True,30.6153935,50.46384579999999
...,...,...,...,...,...,

In [60]:
final_df_2 = final_df.copy()

In [65]:
final_df_2.drop_duplicates(subset=['ИдентификаторЛота'], inplace=True)

In [66]:
final_df_2.Longitude.value_counts()

30.42124179999999     542
34.77950149999999     305
30.6153935            304
27.0071993            284
30.5158454            275
                     ... 
30.736220399999997      1
22.2813892              1
32.0222306              1
30.3722885              1
23.9891794              1
Name: Longitude, Length: 3674, dtype: int64

In [70]:
final_df_2[final_df_2['Longitude'].str.contains('30.42124179999999', na=False)]

,id,city,Назва закладу,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator,Заклад-замовник,Longitude,Latitude
3114,"Київ Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",Київ,"Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",UA-2017-01-13-000699-a-L1,UA-2017-01-13-000699-a,"Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",Надання послуг охорони за допомогою технічних засобів (обслуговування охоронної сигналізації -виїзд групи швидкого реагування).,6000,". Посадові особи замовника, уповноважені здійснювати зв’язок з учасниками: Богуш Олександр Васильович, заступник директора,тел./факс (044) 408-25-18, (050-352-99-86)",Надання послуг охорони за допомогою технічних засобів (обслуговування охоронної сигналізації -виїзд групи швидкого реагування).,"бульвар Гавела Вацлава, 46","бульвар Гавела Вацлава, 46",КМДА,23516367,True,30.42124179999999,50.4239213
3115,"Київ Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",Київ,"Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",UA-2017-03-27-000750-b-L1,UA-2017-03-27-000750-b,"Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",Карнизи пластикові та крючки до них,21000,"Посадові особи замовника, уповноважені здійснювати зв’язок з учасниками : \n Ковальчук Ольга Михайлівна, заст. директора, тел./факс тел.0678972088",Карнизи пластикові та крючки до них,"бульвар Гавела Вацлава, 46","бульвар Гавела Вацлава, 46",КМДА,23516367,True,30.42124179999999,50.4239213
3116,"Київ Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",Київ,"Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",UA-2017-02-01-002645-b-L1,UA-2017-02-01-002645-b,"Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",Вироби фармацевтичні (ліки),50000,"Посадові особи замовника, уповноважені здійснювати зв’язок з учасниками : Клюзко Людмила Володим...",Вироби фармацевтичні (ліки),"бульвар Гавела Вацлава, 46","бульвар Гавела Вацлава, 46",КМДА,23516367,True,30.42124179999999,50.4239213
3117,"Київ Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",Київ,"Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",UA-2017-03-09-001072-a-L1,UA-2017-03-09-001072-a,"Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",Бензин моторний А-95(талони).,67000,"Посадові особи замовника, уповноважені здійснювати зв’язок з учасниками:Богуш Олександр Васильович - заступник директора, тел./факс (044) 408-25-18, (050-352-99-86)",Бензин моторний А-95(талони).,"бульвар Гавела Вацлава, 46","бульвар Гавела Вацлава, 46",КМДА,23516367,True,30.42124179999999,50.4239213
3118,"Київ Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",Київ,"Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",UA-2017-10-06-001362-c-L1,UA-2017-10-06-001362-c,"Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",Медичний м’яч - МЕДБОЛ,18500,"1.\t\tМ’яч медбол: вага – 1 кг (Ø14 см)\tШт.\t40\t350,00\t-14 000,00\n2.\t\tМ’яч медбол: вага – 2 кг (Ø17 см)\tШт.\t10\t450,00\t-4 500,00",Медичний м’яч - МЕДБОЛ,бул. Гавела Вацлава-46,"бульвар Гавела Вацлава, 46",КМДА,23516367,True,30.42124179999999,50.4239213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651,"Київ Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",Київ,"Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",UA-2019-04-11-000776-c-L1,UA-2019-04-11-000776-c,"Загальноосвітній навчальний заклад ""Київський спортивний ліцей-інтернат""",Харчоварильний котел.,40000,Харчоварильний котел.,Харчоварильний котел.,Бульвар Вацлава Гавела -46,"бульвар Гавела Вацлава, 46",КМДА,23516367,True,30.42124179999999,50.4239213
3652,"Київ За

In [74]:
final_df_2.id[final_df_2['Longitude'].str.contains('30.5158454', na=False)].unique()

array(['Київ Муніципальний заклад вищої освіти "Київська Академія мистецтв"'],
      dtype=object)

In [75]:
final_df_2.shape

(49541, 17)